In [2]:
%pip install pycaret
%pip install fsspec openpyxl


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pycaret.classification import *
from pycaret.regression import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
pm_data = pd.read_csv("clean_data\\cleaned_data.csv")
pm_data = pm_data.drop(["Unnamed: 0"],axis=1)
pm_data["timestamp"] = pd.to_datetime(pm_data["timestamp"])
print(pm_data)

               timestamp   humidity     pm_2_5  temperature
0    2021-08-16 07:00:00  97.712066   5.979462    24.220945
1    2021-08-17 07:00:00  95.922874   9.142292    24.807743
2    2021-08-18 07:00:00  87.291672   7.901869    26.346601
3    2021-08-19 07:00:00  86.124131  10.989553    27.062054
4    2021-08-20 07:00:00  89.532874   8.201077    26.208938
...                  ...        ...        ...          ...
1995 2021-10-17 06:00:00  91.460667   0.017094    22.848063
1996 2021-10-17 07:00:00  89.727773   0.008621    22.995754
1997 2021-10-17 08:00:00  86.384701   0.000000    23.557021
1998 2021-10-17 09:00:00  85.498444   0.025641    23.630604
1999 2021-10-17 10:00:00  77.589990   0.000000    25.050899

[2000 rows x 4 columns]


In [6]:
s = setup(pm_data, target = 'pm_2_5', session_id = 123,date_features=["timestamp"],normalize=True)

,Description,Value
0,Session id,123
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(2000, 4)"
4,Transformed data shape,"(2000, 6)"
5,Transformed train set shape,"(1400, 6)"
6,Transformed test set shape,"(600, 6)"
7,Numeric features,2
8,Date features,1
9,Preprocess,True


In [7]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,20.4627,21429.8678,127.8575,0.9041,0.7986,1.8374,0.1130
rf,Random Forest Regressor,18.0329,29796.6173,136.8624,0.8643,0.6097,1.6691,0.1570
lightgbm,Light Gradient Boosting Machine,35.8239,35791.3015,159.4172,0.8260,1.2742,5.2662,0.0900
ada,AdaBoost Regressor,41.1019,48881.4032,211.1542,0.8210,1.0965,5.7758,0.0360
gbr,Gradient Boosting Regressor,35.7269,38241.9933,169.9641,0.8128,1.2891,8.5674,0.0790
dt,Decision Tree Regressor,15.9361,38596.9190,128.9697,0.8050,0.6774,1.6292,0.0270
knn,K Neighbors Regressor,52.7906,72374.2307,260.5418,0.7537,1.0480,9.4041,0.0270
br,Bayesian Ridge,193.3358,231893.0987,477.5750,0.1837,2.4724,37.7198,0.0210
llar,Lasso Least Angle Regression,193.4383,231884.1173,477.5672,0.1836,2.4745,37.6490,0.0220
lasso,Lasso Regression,193.4354,231884.2603,477.5671,0.1836,2.4745,37.6525,0.0240


In [8]:
best_model = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.9097,66602.7467,258.0751,0.5712,0.7285,1.0106
1,28.6581,33877.7597,184.0591,0.9251,0.6504,1.0524
2,13.8153,3411.1935,58.4054,0.9910,0.7869,1.4677
3,24.5915,24680.6571,157.1008,0.9173,0.5623,1.5266
4,6.9764,301.1888,17.3548,0.9981,0.9119,1.0409
5,15.8908,3200.8351,56.5759,0.9929,0.8514,3.9883
6,11.7851,4502.2806,67.0990,0.9853,0.4796,0.7092
7,24.0523,32518.5896,180.3291,0.7909,0.9394,2.5366
8,24.1436,26452.9773,162.6437,0.9305,1.0233,1.9464


In [9]:
best_model_tune = tune_model(best_model)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [10]:
ensem_best_model = ensemble_model(best_model_tune, n_estimators = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.8201,64567.8287,254.1020,0.5843,0.8066,1.0931
1,31.2094,32646.9128,180.6846,0.9278,0.8706,1.3481
2,22.7916,8026.0377,89.5882,0.9789,0.9493,2.1111
3,28.6326,32709.8811,180.8587,0.8905,0.6724,2.1168
4,13.7372,2674.9978,51.7204,0.9828,1.1380,1.6147
5,27.3238,9947.4275,99.7368,0.9780,1.0992,6.1983
6,18.6391,11984.2071,109.4724,0.9609,0.6168,1.3611
7,30.6751,38168.9372,195.3687,0.7545,1.2335,4.1573
8,32.8782,36034.8502,189.8285,0.9053,1.2873,2.5416
